# 🚀 Showcase: 2D to 360° Video Generation (Final, Corrected Version)

This notebook provides a comprehensive and robust showcase for the 'Beyond the Frame' project. It has been redesigned to be as reliable as possible, with a self-correcting setup process.

### **Instructions**

1.  **GPU Runtime:** Ensure your runtime is set to use a GPU (`Runtime` > `Change runtime type` > `T4 GPU`).
2.  **Run Cells in Order:** Execute each cell from top to bottom. There are only two main steps required before you can run the demo.

### **Step 1: Install Conda & Restart Kernel**

⚠️ **CRITICAL STEP** ⚠️

Running the cell below will install `conda` and **automatically restart the Colab kernel.** This is expected and necessary.

**After the kernel restarts, you must continue running the cells from Step 2.**

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

--- 
### **Step 2: Full Setup & Verification (Run This After Kernel Restart)**

**IMPORTANT:** Run this single cell *after* the kernel has restarted from Step 1. 

This cell is **idempotent**, meaning it can be run multiple times without causing errors. It will automatically clean up any previous runs before starting. It will:
1.  Delete any old project files.
2.  Clone the project repository.
3.  Install all dependencies in the correct order.
4.  Download all required AI models from their correct sources.
5.  Verify that all files are in the correct place.

In [ ]:
import os
import sys
import subprocess

# --- Start of Setup ---
print('Starting setup... This may take several minutes.')

# Ensure we are in the root directory and clean up previous runs
if os.path.basename(os.getcwd()) == 'argus-code':
    os.chdir('..')
print('Cleaning up previous runs...')
!rm -rf argus-code
print('✅ Cleanup complete.')

def run_and_verify(command, check_path=None, failure_message=''):
    print(f'--- Running: {failure_message} ---')
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f'❌ FATAL: {failure_message} failed.')
        print(f'--- STDOUT ---\n{result.stdout}')
        print(f'--- STDERR ---\n{result.stderr}')
        sys.exit(1)
    if check_path and not os.path.exists(check_path):
        print(f'❌ FATAL: Verification failed. Expected path does not exist: {check_path}')
        sys.exit(1)
    print(f'✅ {failure_message} successful.')

# --- Step 2.1: Clone Repository ---
run_and_verify('git clone --recurse-submodules https://github.com/Red-Fairy/argus-code', 
                 check_path='argus-code', 
                 failure_message='Cloning repository')
os.chdir('argus-code')

# --- Step 2.2: Install Dependencies ---
run_and_verify('pip install gdown --upgrade', failure_message='Upgrading gdown') # Ensure gdown is recent enough for --folder
run_and_verify('conda install -y -c pytorch -c nvidia faiss-gpu=1.8.0 cython', 
                 failure_message='Conda installation')
run_and_verify('pip install -q -r requirements.txt', 
                 failure_message='Pip requirements installation')
run_and_verify('pip install -q yt-dlp', 
                 failure_message='Pip yt-dlp installation')

# --- Step 2.3: Compile ASMK ---
run_and_verify('git clone https://github.com/jenicek/asmk', 
                 check_path='asmk', 
                 failure_message='Cloning ASMK')
os.chdir('asmk/cython/')
run_and_verify('cythonize *.pyx', 
                 failure_message='Cythonizing ASMK')
os.chdir('..')
run_and_verify('pip install .', 
                 failure_message='Installing ASMK')
os.chdir('..')

# --- Step 2.4: Download Models ---
os.makedirs('checkpoints/pretrained-weights/unet', exist_ok=True)
run_and_verify('gdown --id 1PFRmbD9rLp6HZWobIMMfiasftmyT4xMW -O checkpoints/pretrained-weights/config.json', 
               check_path='checkpoints/pretrained-weights/config.json', 
               failure_message='Downloading config.json')

run_and_verify('gdown --id 1Yq0wEX243L7O2S21zywz2UP-0B53QZz7 -O checkpoints/pretrained-weights/unet/diffusion_pytorch_model.safetensors', 
               check_path='checkpoints/pretrained-weights/unet/diffusion_pytorch_model.safetensors', 
               failure_message='Downloading UNet Safetensors')

os.makedirs('mega-sam/Depth-Anything/checkpoints', exist_ok=True)
run_and_verify('wget -q https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth -O mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth', 
                 check_path='mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth', 
                 failure_message='Downloading DepthAnything model')

os.makedirs('mega-sam/cvd_opt', exist_ok=True)
run_and_verify('wget -q https://huggingface.co/ddrfan/RAFT/resolve/bfb9a87633743408b9c13c20b9ed43dbe5b83617/raft-things.pth -O mega-sam/cvd_opt/raft-things.pth', 
                 check_path='mega-sam/cvd_opt/raft-things.pth', 
                 failure_message='Downloading RAFT model')

# --- Step 2.5: Verify Final Files ---
files_to_check = [
    'checkpoints/pretrained-weights/unet/diffusion_pytorch_model.safetensors',
    'checkpoints/pretrained-weights/config.json',
    'mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth',
    'mega-sam/cvd_opt/raft-things.pth'
]
for f in files_to_check:
    run_and_verify('', check_path=f, failure_message=f'Verifying final file: {f}')
    
print('\n--- ✅✅✅ SETUP COMPLETE AND FULLY VERIFIED ✅✅✅ ---')

### **Step 3: Automated Processing from a URL**

Run the two cells below to automatically download a video from a URL and process it using the command-line script.

In [ ]:
# --- You can change this URL to any video you want to process ---
video_url = 'https://www.youtube.com/watch?v=YE7VzlLtp-4' # Big Buck Bunny (placeholder)

!mkdir -p input_videos
!yt-dlp -o 'input_videos/video.mp4' -f 'best[height<=720]' {video_url}

video_path = 'input_videos/video.mp4'
if os.path.exists(video_path):
    print(f'✅ Video downloaded successfully to: {video_path}')
else:
    print(f'❌ FAILED to download video. The URL may be invalid.')

In [ ]:
if os.path.exists(video_path):
    print('Starting automated processing...')
    unet_path = './checkpoints/pretrained-weights'
    save_folder = './outputs'
    
    !python inference.py \
        --unet_path {unet_path} \
        --video_path {video_path} \
        --val_save_folder {save_folder} \
        --guidance_scale 3 \
        --num_inference_steps 25
    
    print(f'\n--- Processing Complete! ---\n')
    print(f'Check the `{save_folder}` directory for your generated 360° video.')
else:
    print('Skipping processing because video download failed.')